# ABSTRACT

### The journal "Housing Price Prediction with Improved Machine Learning Techniques" has worked on house prices in Beijing city, China. That paper has predicted house prices with not only ensemble methods but also other modern techniques like Hybrid and Stacked Generalization. We will replicate their work to make a case for future work like predicting UK housing prices. 

# 1. INTRODUCTION

### In recent years, due to the growing trend towards Big Data, machine learning has become a vital prediction approach because it can predict house prices more accurately based on their attributes, regardless of the data from previous years. Several studies explored this problem and proved the capability of the machine learning approach. However, most of them compared the models’ performance but did not consider the combination of different machine learning models such as Hybrid regression and Stacked Generalizaiton.

# 2. METHODOLOGY

## 2.1 Data Preprocessing

### We will use the Beijing dataset (taken from Kaggle) comprised of ten years from 2008 to 2018. It has 318851 rows (data) with 26 features. Each feature represents specific information related to housing.

### To work on the data, we have selected Python Language and fetch all the relevant libraries or modules, related to reading the data, finding missing values, exploratory analysis, model designing, and model evaluation.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Data Processing 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# ignore all future warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning) 
simplefilter(action='ignore', category=UserWarning) 

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Satistical Testing
from scipy import stats

# Converting Categorical to Numerical Finite Values
from sklearn.preprocessing import LabelEncoder

# Model Designing and Evaluation
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
import xgboost as xgb
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
# Reading the Data
df =  pd.read_csv('/kaggle/input/lianjia/new.csv',encoding='gbk',low_memory=False)#, errors='ignore')
# Shape of the Data
print ('DATA',df.shape)
df.head(1)

In [ ]:
# Columns or Features or Variables
df.columns

### After reading the data, the first step is to find the missing data and remove it. Eliminating any feature having more than 50% missing data or any row with missing values.

### The second step is to remove unnecessary features in the dataset which has no relevance like a number of "kitchens", "bathrooms", "drawing rooms". Similarly, the data fetched information like "URL", their respective identifications, "Cid" and "id". In the dataset, there are two price features, one is showing the irrelevant price or false data and the other one is the housing price. Therefore, we will remove the "totalPrice" feature too. 


In [ ]:
# Finding missing Values in each column or feature
df.isnull().sum()

### After finding the missing values of each column, it is clear that the feature "DOM"(Day on Market) has missing data almost 50%. We will drop this feature along with missing rows.

### Also removing the unnecessary features as said above. Their names are "url", "kitchen", "drawingRoom", "bathRoom", "Cid", "id", and "totalPrice". 

In [ ]:
# Dropping 50% less values feature and no information features
df.drop(['DOM','url','kitchen','drawingRoom','bathRoom','Cid','id','totalPrice'],axis=1,inplace=True)
# Dropping missing data rows
df.dropna(inplace=True)
df = df[df['constructionTime']!='未知']
# Now the remaining data
print ("DATA", df.shape)

## 2.2 Data Aanalysis

### Data Analysis is the main part before selecting the algorithm on the basis of exploratory analysis. It consists of feature engineering, feature selection, finding outliers and trends through Data visualization. 

### For Feature Engineering, we will create new features according to our requirements. "distance" from latitude and longitude values around Beijing city. This will help to see the prices of houses which are near or far away from the center of Beijing. Feature "Age" showing the age of a house after construction. The year 2019 is set as a threshold. By creating this feature, we don't have to use the feature's 'construction time'. "timeTrade" feature is changed to only year base. 

### Splitting the feature "floor" into two features "floor type" and "floor height" with some data operations like replacing Chinese language words with English words. It will also provide a reflection on the housing price. 

### Replacing numeric values with appropriate house type title in "buildingType" feature. This will help to see the importance of each building aganist price value. Later we will change the values to numeric for model simulation.

In [ ]:
# Creating 'distance' feature
# To calculate Distance Between Two Points on Earth 
from math import radians, cos, sin, asin, sqrt
# We will find distance agnaist each lat and lng from Beijing (lat:39.916668,lon:116.383331)
def distance(lat2, lon2,lat1=39.916668,lon1=116.383331): 
      
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
       
    # calculate the result 
    return(c * r) 
df['distance'] = df.apply(lambda x: distance (x['Lat'],x['Lng']),axis=1)

# Creating "Age" feature by substrating from the threshold value "2019"
df['constructionTime'] = df['constructionTime'].astype(int)
df['Age'] = 2019 - df['constructionTime']

# 'timeTrade' feature to year base only.
df['tradeTime'] = pd.DatetimeIndex(df['tradeTime']).year

# Creating "floor type" and "floor height" features
# Fisrt split the string (floor type) and numeric part (the height)
lst_numeric = []
lst_str = []
for value in df['floor'].values:
    value = value.split()
    numeric = (value[1])
    string  = value[0]
    lst_numeric.append(numeric)
    lst_str.append(string)

# Replacing Chinese language words with English words.    
lst_str_eng=[]
for string in lst_str:
    #print(string)
    if string == '中':
        lst_str_eng.append(string.replace('中','middle'))
    elif string == '高':
        lst_str_eng.append(string.replace('高','high'))
    elif string == '底':
        lst_str_eng.append(string.replace('底','bottom'))
    elif string == '低':
        lst_str_eng.append(string.replace('低','low'))
    elif string == '未知':
        lst_str_eng.append(string.replace('未知','unknown'))
    elif string == '顶':
        lst_str_eng.append(string.replace('顶','top'))

#print (len(lst_str_eng))
# Converting intto Data Frame or in one shape dataset
df1 = pd.DataFrame(lst_str_eng,columns=['floorType'])
df2 = pd.DataFrame(lst_numeric,columns=['floorHeight'])
df = pd.concat([df,df1,df2],axis=1)
# Deleting unknown values
df = df[df['floorType']!='unknown']

# Dropping missing data which can't be converted into real data
df.dropna(inplace=True)

# Dropping features which are not much relevant now.
df.drop(['floor','constructionTime'],axis=1,inplace=True)

# Converting 'buildingType' feature to object or string type
df['buildingType'].replace(1,'Tower',inplace=True)
df['buildingType'].replace(2,'Bungalow',inplace=True)
df['buildingType'].replace(3,'Tower and Plate',inplace=True)
df['buildingType'].replace(4,'Plate',inplace=True)


# Converting features datatype to see outliers
df['floorHeight'] = df['floorHeight'].astype(int)
df['livingRoom'] = df['livingRoom'].astype(int)
df['district'] = df['district'].astype(int)
df['tradeTime'] = df['tradeTime'].astype(int)
df['Age'] = df['Age'].astype(int)
df['renovationCondition'] = df['renovationCondition'].astype(int)
df['buildingStructure'] = df['buildingStructure'].astype(int)
df['elevator'] = df['elevator'].astype(int)
df['fiveYearsProperty'] = df['fiveYearsProperty'].astype(int)
df['subway'] = df['subway'].astype(int)
df['followers']  = df['followers'].astype(int)
# Reseting the index
df.reset_index(inplace=True)
df.drop(['index'],axis=1,inplace=True)
# Now the remaining data
print ("DATA", df.shape)
df.head()

### After creating new features, their relationship with housing prices along with related features can easily see through visualizations.

### Visualization is a great technique to find correlation and outliers.

In [ ]:
# Distance Impact on the Price

sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.scatterplot(data=df, x="Lat", y="Lng", hue="price",size = 'distance',
                     legend='brief')
ax.set(xlabel="Latitude", ylabel = "Longitude",title = "Price Distribution Distance Wise")

In [ ]:
# District wise Price

sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.scatterplot(data=df, x="Lat", y="Lng", hue="price",size = 'district',
                     legend='brief')
ax.set(xlabel="Latitude", ylabel = "Longitude",title = "Price Distribution District Wise")

In [ ]:
# Age wise Distribution

sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.scatterplot(data=df, x="Lat", y="Lng", hue="Age",size = 'district',
                     legend='brief')
ax.set(xlabel="Latitude", ylabel = "Longitude",title = "Age Distributio District Wise")

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.boxplot(data=df, x="district", y="price")
ax.set(xlabel="Districts", ylabel = "Price",title = "Correlation between District and Price")

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.boxplot(data=df, x="buildingType", y="price")
ax.set(xlabel="Building Type", ylabel = "Price",title = "Correlation between Building Type and Price")

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.boxplot(data=df, x="buildingType", y="square")
ax.set(xlabel="Building Type", ylabel = "Square",title = "Correlation between Building Type and Square")

### After creating visualizations, the impact on the prices is visible.

### In figures "Price Distribution Distance and District wise", the most expensive houses centralizes close to the center of Beijing, while the cheapest ones spread in the suburban periphery

### In figure "Age Distribution District Wise", the oldest houses are concentrated densely in the center of Beijing, while the newest ones are spread sparsely in the suburban areas.

### Since the patterns in these figures are similar, a strong correlation between location, age, and price can be observed. 

### There are also noticeable differences in housing prices across 13 districts, which are summarized in Fig. "Correlation between District and Price" with outliers are clearly visible

### Besides the location features, other features of the house also significantly contribute to the models’ performance. In the Correlation between BuildingType and Price figure, the difference in price among several building types can be clearly illustrated. Since bungalow is an old building type and is more likely to be built close to the center of Beijing, its price is costly regardless of the small area of a house ( correlation between BuildingType and Square figure)

### After feature engineering, now the dataset is checked for outliers. 

### There are 10 categorical features, and 9 are numerical ones.

### We have to visualize each numerical feature through "Box Plot" graph to see outliers first. Then apply the equation (1) to exclude them 

### Through Inter-Quartile Range (IQR), an outlier x can be detected if:
###          x < Q1 − 1.5 · IQR OR Q3 + 1.5 · IQR < x        (1)
### where:
###       Q1 = 25th percentiles Q3 = 75th percentiles IQR = Q3 − Q1

### To find percentile values, we will find the stats summary through pandas function, applied on each feature.

### Then we will apply equation (1) to every numerical feature of the dataset except the target feature.



### 1. "Lng" Longitude of the House

In [ ]:
df.boxplot(['Lng'])

In [ ]:
df['Lng'].describe()

In [ ]:
# For 'Lng' Outliers
outliers_Lng = []
Q1 = 116.344557
Q3 = 116.481385
IQR = Q3 - Q1
for x in df['Lng'].values:
    if (x < (Q1 - 1.5 * IQR)) or ((Q3 + 1.5 * IQR) < x):
        if x not in outliers_Lng:
            outliers_Lng.append(x)

#print (sorted(outliers_Lng))
for outlier in outliers_Lng:
    df = df[df['Lng']!=outlier]
print ("DATA", df.shape)

### 2. "Lat" Latitude of the House

In [ ]:
df.boxplot('Lat')

In [ ]:
df['Lat'].describe()

In [ ]:
# For 'Lat' Outliers
outliers_Lat = []
Q1 = 39.894045
Q3 = 40.012518
IQR = Q3 - Q1
for x in df['Lat'].values:
    if (x < (Q1 - 1.5 * IQR)) or ((Q3 + 1.5 * IQR) < x):
        if x not in outliers_Lat:
            outliers_Lat.append(x)

#print (sorted(outliers_Lat))
for outlier in outliers_Lat:
    df = df[df['Lat']!=outlier]
print ('DATA', df.shape)

### 3. "distance" Distance to the Center of Beijing

In [ ]:
df.boxplot(['distance'])

In [ ]:
df['distance'].describe()

In [ ]:
# For 'distance' Outliers
outliers_dist = []
Q1 =  7.821041
Q3 = 17.444622
IQR = Q3 - Q1
for x in df['distance'].values:
    if (x < (Q1 - 1.5 * IQR)) or ((Q3 + 1.5 * IQR) < x):
        if x not in outliers_dist:
            outliers_dist.append(x)

#print (sorted(outliers_dist))
for outlier in outliers_dist:
    df = df[df['distance']!=outlier]
print ('DATA' ,df.shape)

### 4. "Age" Age of the House

In [ ]:
df.boxplot('Age')

In [ ]:
df['Age'].describe()

In [ ]:
# For 'Age' Outliers
outliers_age = []
Q1 = 13
Q3 = 25
IQR = Q3 - Q1
for x in df['Age'].values:
    if (x < (Q1 - 1.5 * IQR)) or ((Q3 + 1.5 * IQR) < x):
        if x not in outliers_age:
            outliers_age.append(x)

#print (sorted(outliers_age))
for outlier in outliers_age:
    df = df[df['Age']!=outlier]
print ("DATA" ,df.shape)

### 5. "square" Area of the House

In [ ]:
df.boxplot('square')

In [ ]:
df['square'].describe()

In [ ]:
# For 'square' Outliers
outliers_square = []
Q1 = 58.280000
Q3 = 99.330000
IQR = Q3 - Q1
for x in df['square'].values:
    if (x < (Q1 - 1.5 * IQR)) or ((Q3 + 1.5 * IQR) < x):
        if x not in outliers_square:
            outliers_square.append(x)

#print (sorted(outliers_square))
for outlier in outliers_square:
    df = df[df['square']!=outlier]
print ('DATA', df.shape)

### 6. "Community Average" Average Housing Price of the Community

In [ ]:
df.boxplot('communityAverage')

In [ ]:
df['communityAverage'].describe()

In [ ]:
# For 'communityAverage' Outliers
outliers_cA = []
Q1 = 47402.000000
Q3 = 72877.000000
IQR = Q3 - Q1
for x in df['communityAverage'].values:
    if (x < (Q1 - 1.5 * IQR)) or ((Q3 + 1.5 * IQR) < x):
        if x not in outliers_cA:
            outliers_cA.append(x)

#print (sorted(outliers_cA))
for outlier in outliers_cA:
    df = df[df['communityAverage']!=outlier]
print ('DATA', df.shape)

### 7. "followers" Number of Followers

In [ ]:
df.boxplot(['followers'])

In [ ]:
df['followers'].describe()

In [ ]:
# For 'followers' Outliers
outliers_followers = []
Q1 = 0
Q3 = 20
IQR = Q3 - Q1
for x in df['followers'].values:
    if (x < (Q1 - 1.5 * IQR)) or ((Q3 + 1.5 * IQR) < x):
        if x not in outliers_followers:
            outliers_followers.append(x)

#print (sorted(outliers_followers))
for outlier in  outliers_followers:
    df = df[df['followers']!=outlier]
print ('DATA', df.shape)

### 8. "tradeTime" Trade Time (2002-2018)

In [ ]:
df.boxplot(['tradeTime'])

In [ ]:
df['tradeTime'].describe()

In [ ]:
# For 'followers' Outliers
outliers_tradeTime = []
Q1 = 2013
Q3 = 2016
IQR = Q3 - Q1
for x in df['tradeTime'].values:
    if (x < (Q1 - 1.5 * IQR)) or ((Q3 + 1.5 * IQR) < x):
        if x not in outliers_tradeTime:
            outliers_tradeTime.append(x)

#print (sorted(outliers_tradeTime))
df = df[df['tradeTime'] != 2002]
print ('DATA',df.shape)

### 9. ladderRatio" Ratio b/w population and # of elevators of floor

In [ ]:
df.boxplot(['ladderRatio'])

In [ ]:
df['ladderRatio'].describe()

In [ ]:
# For 'ladderRatio' Outliers
outliers_ladderRatio = []
Q1 = 2.500000e-01
Q3 = 5.000000e-01
IQR = Q3 - Q1
for x in df['ladderRatio'].values:
    if (x < (Q1 - 1.5 * IQR)) or ((Q3 + 1.5 * IQR) < x):
        if x not in outliers_ladderRatio:
            outliers_ladderRatio.append(x)

#print (sorted(outliers_ladderRatio))
for outlier in outliers_ladderRatio:
    df = df[df['ladderRatio']!=outlier]
print ('DATA', df.shape)

### As we know that there are 10 categorical features in the dataset. These should be converted to numerical finite values. Machine Learning algorithms only deal with numerical values. Giving finite values to these features will act in the same manner as a categorical feature. There will be no difference. The categorical features are

### 1. "livingRoom" Number of bedrooms
### 2. "subway" Whether the house is near any subways
### 3. "fiveYearsProperty" Whether the house is a five-year property
### 4. "elevator" Whether the house has any elevator
### 5. "floorHeight" The Floor Height"
### 6. "buildingStructure" Building Structure
### 7. "renovationCondition" Renovation Condition
### 8. "buildingType" Building Type
### 9. "floorType" Floor height relative to the building
### 10. "district" District (District 1- District 13)

### Some are already in numeric form while others have to be transformed.

In [ ]:
# Label Encoding of Categorical Features
lbl = LabelEncoder()
# 'floorType'
lbl.fit(list(df['floorType'].values) + list(df['floorType'].values))
df['floorType'] = lbl.transform(list(df['floorType'].values))
# 'buildingType'
lbl.fit(list(df['buildingType'].values) + list(df['buildingType'].values))
df['buildingType'] = lbl.transform(list(df['buildingType'].values))

### Now all are converted. Lets check data type of each feature for algorithm and the remaining data (219271) after excluding all the outliers

In [ ]:
print ( "The DATA" , df.shape)
df.dtypes

### The Final Data

### According to the paper, their data after omitting outliers contained 231962 data of rows with 19 features. Though we collected 219271 data of rows after performing all the operations. The error is 5% which is acceptable due to any reason like the different approach of the researcher towards data handling in some parts of the data which we considered differently.   

### Features Selection

### Selecting features is always based on statistical information. That's why we have to check the correlation as well as the p-value between the target feature "price" and all the other features.

### The statistical Analysis provides the statistical significance of any feature (the independent variables) for the target or dependent variable. 

### In statistics, "the p-value is the probability of obtaining results at least as extreme as the observed results of a statistical hypothesis test, assuming that the null hypothesis is correct. A smaller p-value means that there is stronger evidence in favor of the alternative hypothesis".


### P-Value is the threshold to check the significance. If P-value < 0.01 or 0.05 or 0.1 (99% or 95% or 90% Significance Levels ) of any independent variable, it is become statistically significant to affect the outcome. In our case, any statistically significant variable will affect the Housing Price. 

### There are mathematical (Stats library SciPy) and visualization (Heat Map) ways to determine the p-value and the correlation like strong, modest, or weak relation either positive or negative relationship with the predicted variables.

In [ ]:
### View of Correlation Matrix Through head map
corr_matrix=df._get_numeric_data().corr()
fig, ax = plt.subplots(figsize=(10,5))         # Sample figsize in inches
sns.heatmap(corr_matrix, annot=False, linewidths=5, ax=ax, xticklabels=corr_matrix.columns.values,yticklabels=corr_matrix.columns.values)
#sns.heatmap(corr, annot=True, fmt=".1f",linewidth=0.5 xticklabels=corr.columns.values,yticklabels=corr.columns.values)

In [ ]:
# finding correlation and p_value
pearson_coef, p_value = stats.pearsonr(df['Lng'], df['price'])
print(" The Pearson Correlation Coefficient 'Lng' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['Lat'], df['price'])
print(" The Pearson Correlation Coefficient 'Lat' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['tradeTime'], df['price'])
print(" The Pearson Correlation Coefficient 'tradeTime' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['followers'], df['price'])
print(" The Pearson Correlation Coefficient 'followers' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['square'], df['price'])
print(" The Pearson Correlation Coefficient 'square' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['livingRoom'], df['price'])
print(" The Pearson Correlation Coefficient 'livingRoom' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['buildingType'], df['price'])
print(" The Pearson Correlation Coefficient 'buildingType' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['renovationCondition'], df['price'])
print(" The Pearson Correlation Coefficient 'renovationCondition' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['buildingStructure'], df['price'])
print(" The Pearson Correlation Coefficient 'buildingStructure' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['ladderRatio'], df['price'])
print(" The Pearson Correlation Coefficient 'ladderRatio' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['elevator'], df['price'])
print(" The Pearson Correlation Coefficient 'elevator' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['fiveYearsProperty'], df['price'])
print(" The Pearson Correlation Coefficient 'fiveYearsProperty' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['communityAverage'], df['price'])
print(" The Pearson Correlation Coefficient 'communityAverage' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df[ 'distance'], df['price'])
print(" The Pearson Correlation Coefficient  'distance' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['Age'], df['price'])
print(" The Pearson Correlation Coefficient 'Age' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['subway'], df['price'])
print(" The Pearson Correlation Coefficient 'subway' is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['district'], df['price'])
print(" The Pearson Correlation Coefficient 'district'is", pearson_coef, " with a P-value of P =", p_value)
pearson_coef, p_value = stats.pearsonr(df['floorHeight'], df['price'])
print(" The Pearson Correlation Coefficient 'floorHeight' is", pearson_coef, " with a P-value of P =", p_value)


### Through visualization and statistical findings; most features have a very small p-value which indicates the null hypothesis is true. Or in other words, they are highly significant.

### Though the correlation is weak for most of the features, but due to dealing with a large sample and keep in mind the p-values, they can be statistically significant.

### Therefore we will pick all the 19 independent features in the dataset to predict the housing prices as did by the researcher.

## 2.3 Model Selection & Evaluation

### The algorithms are dealing with two types of methods, Regression and Classification. As we are predicting prices, it means we will go for regression models.  We will select five different algorithms such as "Random Forest", "Extreme Gradient Boosting (XGBoost)", "Light Gradient Boosting Machine (LightGBM)", "Hybrid Regression" and "Stacked Generalization". The purpose of selecting more than one algorithm is to verify the accuracy across each model to ensure good predicting results.

### After predicting results, the evaluation is a must to put a stamp on your work. We will use the "Root Mean Squared Logarithmic Error (RMSLE)" evaluation technique to check the results. 

### RMSLE is robust to outliers. It only considers the relative error between the Predicted and the actual value, and the scale of the error is not significant. RMSLE incurs a larger penalty for the underestimation of the Actual variable than the Overestimation. In simple words, more penalty is incurred when the predicted value is less than the Actual Value. On the other hand, a Less penalty is incurred when the predicted value is more than the actual value.

## 2.3.1 Random Forest Classifier

### A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is controlled with the max_samples parameter if bootstrap=True (default), otherwise the whole dataset is used to build each tree. 

### The advantages of random forest are

### 1. Decorrelate trees
### 2. Error reducer
### 3. Good performer on imbalanced dataset
### 4. Good in Handling the huge amount of data
### 5. Little impact of outliers
### 6. Robust to overfitting

### Similarly the disadvantages are

### 1. Features need to have some predictive power else they won’t work.
### 2. Predictions of the trees need to be uncorrelated.
### 3. Appears as Black Box: It is tough to know what is happening. You can at best try different parameters and random seeds to change the outcomes and performance.

In [ ]:
# All the columns
df.columns

In [ ]:
# Designing the Base For Each algorithm

# Feature Selection

X = df[['Lng', 'Lat', 'tradeTime', 'followers', 'square', 'livingRoom',
       'buildingType', 'renovationCondition', 'buildingStructure',
       'ladderRatio', 'elevator', 'fiveYearsProperty', 'subway', 'district',
       'communityAverage', 'distance', 'Age', 'floorType', 'floorHeight']]

# Standardized the Features
X = np.asarray(X)
X = preprocessing.StandardScaler().fit(X).transform(X)  

# Selecting Target Feature
y =  np.asarray(df['price'])#.reshape(-1, 1)

# Split the data into training and test data to see results on the test data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Further spliting the Traning set into sub Train and Test Data to see results on training data
train_X,test_X,train_y,test_y = train_test_split(X_train,y_train,random_state=42)


In [ ]:
%%time
# Model Trainng For Training Set
# Specify Model
rf = RandomForestRegressor(random_state=42,n_estimators=900,max_depth=20,
                                              n_jobs=-1,min_samples_split=10,) #110,325
# Fit Model

rf.fit(train_X, train_y.ravel())

# Make validation predictions

pred_y = rf.predict(test_X)

# Model Evaluation

print ("RMSLE {:,.5f}".format(np.sqrt(mean_squared_log_error( pred_y, test_y))))
#print ("Model Accuracy {:,.5f}".format(rf.score(X_test, y_test)))

### The error is low (0.292), it means the algorithm is not working badly. We can ensure the results through visualization too.

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.regplot(pred_y,test_y,color = 'blue',line_kws={'color':'orange'})#,hue='y_test')

ax.set(xlabel="Random Forest Prediction", ylabel = "Actual Price",title = "RANDOM FOREST RESULTS")
ax.legend(['Line of Best Fit', 'Housing Price'], loc='lower right')

### It is clear from the above figure that both prices are directly proportional. So the predicted results are good for the training data.

In [ ]:
%%time
# Model Trainng For Test Data
# Specify Model
rf = RandomForestRegressor(random_state=42,n_estimators=900,max_depth=20,
                                              n_jobs=-1,min_samples_split=10,) #110,325
# Fit Model

rf.fit(X_train, y_train.ravel())

# Make validation predictions

y_pred = rf.predict(X_test)

# Model Evaluation

print ("RMSLE {:,.5f}".format(np.sqrt(mean_squared_log_error( y_pred, y_test))))
#print ("Model Accuracy {:,.5f}".format(rf.score(X_test, y_test)))

### The error on the test data is also low (0.302), it means the algorithm is not working badly. We can ensure the results through visualization 

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.regplot(y_pred,y_test,color = 'blue',line_kws={'color':'orange'})#,hue='y_test')

ax.set(xlabel="Random Forest Prediction", ylabel = "Actual Price",title = "RANDOM FOREST RESULTS")
ax.legend(['Line of Best Fit', 'Housing Price'], loc='lower right')

### It is clear from the above figure that both prices are directly proportional. So the predicted results are good for the test data.

## 2.3.2 Light Gradient Boosting Machine (LightGBM)

### LightGBM uses a novel technique of Gradient-based One-Side Sampling (GOSS) to filter out the data instances for finding a split value

### Advantages of Light GBM

### Faster training speed and higher efficiency: Light GBM use histogram based algorithm i.e it buckets continuous feature values into discrete bins which fasten the training procedure.

### Lower memory usage: Replaces continuous values to discrete bins which result in lower memory usage.

### Better accuracy than any other boosting algorithm: It produces much more complex trees by following leaf wise split approach rather than a level-wise approach which is the main factor in achieving higher accuracy. However, it can sometimes lead to overfitting which can be avoided by setting the max_depth parameter.

### Compatibility with Large Datasets: It is capable of performing equally good with large datasets with a significant reduction in training time as compared to XGBOOST.

### Disadvantages of Light GBM

### Boosting models are more prone to over-fitting , bagging models on other hand is simple to use and doesn't have lot of hyper parameter.


In [ ]:
%%time
# Light GBM Model Training For Training Data

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='regression',num_leaves=36,learning_rate=0.15,
                        n_estimators=64,min_child_weight = 2, colsample_bytree = 0.8,
                        reg_lambda = 0.45)
gbm.fit(train_X, train_y,
    eval_set=[(test_X, test_y)],
    eval_metric='l2_root',
    early_stopping_rounds=50)
pred_y = gbm.predict(test_X, num_iteration=gbm.best_iteration_)
#accuracy = round(gbm.score(X_train, y_train)*100,2)
#mse = mean_squared_error(y_test,y_pred)
#rmse = np.sqrt(mse)
print ("RMSLE {:,.5f}".format(np.sqrt(mean_squared_log_error( pred_y, test_y))))

### The error is low (0.297), it means the algorithm is not working badly. We can ensure the results through visualization 

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.regplot(pred_y,test_y,color = 'blue',line_kws={'color':'orange'})#,hue='y_test')

ax.set(xlabel="Light GBM Prediction", ylabel = "Actual Price",title = "Light GBM RESULTS")
ax.legend(['Line of Best Fit', 'Housing Price'], loc='lower right')

### It is clear from the above figure that both prices are directly proportional. So the predicted results are good for the training data.

In [ ]:
%%time
# Light GBM Model Training For Test Data

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='regression',num_leaves=36,learning_rate=0.15,
                        n_estimators=64,min_child_weight = 2, colsample_bytree = 0.8,
                        reg_lambda = 0.45)
gbm.fit(X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='l2_root',
    early_stopping_rounds=50)
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
#accuracy = round(gbm.score(X_train, y_train)*100,2)
#mse = mean_squared_error(y_test,y_pred)
#rmse = np.sqrt(mse)
print ("RMSLE {:,.5f}".format(np.sqrt(mean_squared_log_error( y_pred, y_test))))

### The error on the test data is also low (0.308), it means the algorithm is not working badly. We can ensure the results through visualization. 

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.regplot(y_pred,y_test,color = 'blue',line_kws={'color':'orange'})#,hue='y_test')

ax.set(xlabel="Light GBM Prediction", ylabel = "Actual Price",title = "Light GBM RESULTS")
ax.legend(['Line of Best Fit', 'Housing Price'], loc='lower right')

### It is clear from the above figure that both prices are directly proportional. So the predicted results are good for the test data.

## 2.3.3 Extreme Gradient Boosting (XGBoost)

### XGBoost is a scalable machine learning system for tree boosting. The system is available as an open-source package.The system has generated a significant impact and been widely recognized in various machine learning and data mining challenges.

# PROS

### Less feature engineering required (No need for scaling, normalizing data, can also handle missing values well)

### Feature importance can be found out(it output importance of each feature, can be used for feature selection)

### Fast to interpret

### Outliers have minimal impact.

### Handles large sized datasets well.

### Good Execution speed

### Good model performance (wins most of the Kaggle competitions)

### Less prone to overfitting

# CONS

### Difficult interpretation , visualization tough
### Overfitting possible if parameters not tuned properly.
### Harder to tune as there are too many hyperparameters.

In [ ]:
%%time
# XGBoost Model Training For Training Data
# Traing the Model
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', min_child_weight = 2, subsample = 1,
                          colsample_bytree = 0.8,
                          learning_rate = 0.2, n_estimators = 500,
                         reg_lambda = 0.45, reg_alpha = 0, gamma = 0.5)

# Fitting the Model
xg_reg.fit(train_X,train_y)

# Predicting
pred_y = xg_reg.predict(test_X)

# Evaluation
print ("RMSLE {:,.5f}".format(np.sqrt(mean_squared_log_error( pred_y, test_y))))

### The error is low (0.291), it means the algorithm is not working badly. We can ensure the results through visualization 

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.regplot(pred_y,test_y,color = 'blue',line_kws={'color':'orange'})#,hue='y_test')

ax.set(xlabel="XGBoost Prediction", ylabel = "Actual Price",title = "XGBoost RESULTS")
ax.legend(['Line of Best Fit', 'Housing Price'], loc='lower right')

### It is clear from the above figure that both prices are directly proportional. So the predicted results are good for the training data.

In [ ]:
%%time
# XGBoost Model Training For Test Data
# Traing the Model
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', min_child_weight = 2, subsample = 1,
                          colsample_bytree = 0.8,
                          learning_rate = 0.2, n_estimators = 500,
                         reg_lambda = 0.45, reg_alpha = 0, gamma = 0.5)

# Fitting the Model
xg_reg.fit(X_train,y_train)

# Predicting
y_pred = xg_reg.predict(X_test)

# Evaluation
print ("RMSLE {:,.5f}".format(np.sqrt(mean_squared_log_error( y_pred, y_test))))

### The error on the test data is very low (0.302), it means the algorithm is not working badly. We can ensure the results through visualization 

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.regplot(y_pred,y_test,color = 'blue',line_kws={'color':'orange'})#,hue='y_test')

ax.set(xlabel="XGBoost Prediction", ylabel = "Actual Price",title = "XGBoost RESULTS")
ax.legend(['Line of Best Fit', 'Housing Price'], loc='lower right')

### It is clear from the above figure that both prices are directly proportional. So the predicted results are good for the test data.

## 2.3.4 Hybrid Regression

### Hybrid Regression is a model that includes two or more different regression models. We will combine all the above three models to make a Hybrid model. 

### This method averages the individual predictions to form a final prediction. It is a fair approach to balance between bias and variance in the composite models. This technique also supports weight assignment to each component model, but it may lead to bias over one model, losing the benefits of generalization.

###  Hybrid Regression is actually a VotingRegressor algorithm in sklearn module. A voting regressor is an ensemble meta-estimator that fits several base regressors, each on the whole dataset. Then it averages the individual predictions to form a final prediction.

In [ ]:
%%time
# Hybrid MOdel Training For Training Data
# Model 1
r1 = xgb.XGBRegressor(objective ='reg:squarederror', min_child_weight = 2, subsample = 1,
                          colsample_bytree = 0.8,
                          learning_rate = 0.2, n_estimators = 500,
                         reg_lambda = 0.45, reg_alpha = 0, gamma = 0.5)
# Model 2
r2 = lgb.LGBMRegressor(objective='regression',num_leaves=36,learning_rate=0.15,
                        n_estimators=64,min_child_weight = 2, colsample_bytree = 0.8,
                        reg_lambda = 0.45)
# Model 3
r3 = RandomForestRegressor(random_state=42,n_estimators=900,max_depth=20,
                                              n_jobs=-1,min_samples_split=10,)

# Making Hybrid Model
vreg = VotingRegressor([('gb', r1), ('lgb', r2), ('rf', r3)])

# Fitting the Model
vreg.fit(train_X, train_y)

# Predicting the Model
pred_y = vreg.predict(test_X)

# Evaluation
print ("RMSLE {:,.5f}".format(np.sqrt(mean_squared_log_error( pred_y, test_y))))

### The error is low (0.291), it means the algorithm is not working badly. We can ensure the results through visualization

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.regplot(pred_y,test_y,color = 'blue',line_kws={'color':'orange'})#,hue='y_test')

ax.set(xlabel="Hybrid Prediction", ylabel = "Actual Price",title = "Hybrid Results")
ax.legend(['Line of Best Fit', 'Housing Price'], loc='lower right')

### It is clear from the above figure that both prices are directly proportional. So the predicted results are good for the training data.

In [ ]:
%%time
# Hybrid Model Training For Test Data
# Model 1
r1 = xgb.XGBRegressor(objective ='reg:squarederror', min_child_weight = 2, subsample = 1,
                          colsample_bytree = 0.8,
                          learning_rate = 0.2, n_estimators = 500,
                         reg_lambda = 0.45, reg_alpha = 0, gamma = 0.5)
# Model 2
r2 = lgb.LGBMRegressor(objective='regression',num_leaves=36,learning_rate=0.15,
                        n_estimators=64,min_child_weight = 2, colsample_bytree = 0.8,
                        reg_lambda = 0.45)
# Model 3
r3 = RandomForestRegressor(random_state=42,n_estimators=900,max_depth=20,
                                              n_jobs=-1,min_samples_split=10,)

# Making Hybrid Model
vreg = VotingRegressor([('gb', r1), ('lgb', r2), ('rf', r3)])

# Fitting the Model
vreg.fit(X_train, y_train)

# Predicting the Model
y_pred = vreg.predict(X_test)

# Evaluation
print ("RMSLE {:,.5f}".format(np.sqrt(mean_squared_log_error( y_pred, y_test))))

The error is low (0.302), it means the algorithm is not working badly. We can ensure the results through visualization

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.regplot(y_pred,y_test,color = 'blue',line_kws={'color':'orange'})#,hue='y_test')

ax.set(xlabel="Hybrid Prediction", ylabel = "Actual Price",title = "Hybrid Results")
ax.legend(['Line of Best Fit', 'Housing Price'], loc='lower right')

### It is clear from the above figure that both prices are directly proportional. So the predicted results are good for the test data.

## 2.3.5 Stacked Generalization

### Stacked generalization consists in stacking the output of individual estimator and use a regressor to compute the final prediction. Stacking allows to use the strength of each individual estimator by using their output as input of a final estimator.

### The main idea of this method is to use the predictions of previous models as features for another model. This approach also utilizes the k-fold cross-validation technique to avoid overfitting.

### sklearn module StackingRegressor is based on stacked generalization

In [ ]:
%%time
# Stacked Model Training For Training Data

# Model 1
r1 = xgb.XGBRegressor(objective ='reg:squarederror', min_child_weight = 2, subsample = 1,
                          colsample_bytree = 0.8,
                          learning_rate = 0.2, n_estimators = 500,
                         reg_lambda = 0.45, reg_alpha = 0, gamma = 0.5)
# Model 2
r2 = lgb.LGBMRegressor(objective='regression',num_leaves=36,learning_rate=0.15,
                        n_estimators=64,min_child_weight = 2, colsample_bytree = 0.8,
                        reg_lambda = 0.45)
# Model 3
r3 = RandomForestRegressor(random_state=42,n_estimators=900,max_depth=20,
                                              n_jobs=-1,min_samples_split=10,)

# Making Stacked Model
estimators = [('xgb', r1),('lgb', r2)]
sreg = StackingRegressor(estimators=estimators,final_estimator = r3)
                         

# Fitting the Model
sreg.fit(train_X, train_y)

# Predicting the Model
pred_y = sreg.predict(test_X)

# Evaluation
print ("RMSLE {:,.5f}".format(np.sqrt(mean_squared_log_error( pred_y, test_y))))


The error is low (0.293), it means the algorithm is not working badly. We can ensure the results through visualization

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.regplot(pred_y,test_y,color = 'blue',line_kws={'color':'orange'})#,hue='y_test')

ax.set(xlabel="Stacked Prediction", ylabel = "Actual Price",title = "Stacked Results")
ax.legend(['Line of Best Fit', 'Housing Price'], loc='lower right')

It is clear from the above figure that both prices are directly proportional. So the predicted results are good for the training data.

In [ ]:
%%time
# Stacked Model Training For Test Data

# Model 1
r1 = xgb.XGBRegressor(objective ='reg:squarederror', min_child_weight = 2, subsample = 1,
                          colsample_bytree = 0.8,
                          learning_rate = 0.2, n_estimators = 500,
                         reg_lambda = 0.45, reg_alpha = 0, gamma = 0.5)
# Model 2
r2 = lgb.LGBMRegressor(objective='regression',num_leaves=36,learning_rate=0.15,
                        n_estimators=64,min_child_weight = 2, colsample_bytree = 0.8,
                        reg_lambda = 0.45)
# Model 3
r3 = RandomForestRegressor(random_state=42,n_estimators=900,max_depth=20,
                                              n_jobs=-1,min_samples_split=10,)

# Making Stacked Model
estimators = [('xgb', r1),('lgb', r2)]
sreg = StackingRegressor(estimators=estimators,final_estimator = r3)
                         

# Fitting the Model
sreg.fit(X_train, y_train)

# Predicting the Model
y_pred = sreg.predict(X_test)

# Evaluation
print ("RMSLE {:,.5f}".format(np.sqrt(mean_squared_log_error( y_pred, y_test))))


### The error is very low (0.303), it means the algorithm is not working badly. We can ensure the results through visualization

In [ ]:
sns.set(rc={'figure.figsize':(12.7,8.27)})
ax = sns.regplot(y_pred,y_test,color = 'blue',line_kws={'color':'orange'})#,hue='y_test')

ax.set(xlabel="Stacked Prediction", ylabel = "Actual Price",title = "Stacked Results")
ax.legend(['Line of Best Fit', 'Housing Price'], loc='lower right')

### It is clear from the above figure that both prices are directly proportional. So the predicted results are good for the test data.

# 3. Results

## RMSLE

| Model | Train Set | Test Set |
| --- | --- | --- |
| Random Forest | 0.292 | 0.302 |
| Light GBM | 0.297 | 0.308 |
| XG Boost  | 0.291 | 0.302 |
| Hybrid Regression   | 0.291 | 0.302 |
| Stacked Generalization | 0.293 | 0.303 |

## 3.1 Results Explanation

### The difference among each algorithm is very minor. Results are more or less the same.

### XG Boost and Hybrid Regression results are the better for both Train as well as Test Sets.

### Train Set is showing less error as compared to test set, which we can take as some overfitting to some extent.

## 3.2 Results Comparison witht the Researcher Work

### The researcher models are showing almost the same results among them but with less error as compared to our results

### Their minimum Train Set error is 0.13 while we reach at 0.29

### Similarly for Test Set error is 0.16 while our is showing 0.30 

## 3.3 Reasons For Different Results

### We fetched the data from Kaggle source which has more bugs.
### We deleted all the missing values as did by the researcher. But if we replace with appropriate values, then no data will be lost and there is more chance to acquire better results.

# 4. Recommendations

### After replicating the paper, we can improve our results by doing more exploratory analysis.

### We can also apply for estimating any housing price of different cities by keeping in mind the relevant data and the methodology applied in the paper.

### Rough Work
def score_func(y_true, y_pred, **kwargs):
    y_true = np.abs(y_true)
    y_pred = np.abs(y_pred)

    return np.sqrt(mean_squared_log_error(y_true, y_pred))
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

parameters = {'n_estimators':[i for i in range(100,300,100)],'max_depth':[i for i in range(0,4,2)],
              'min_samples_split':[i for i in range(2, 4)],'random_state': [42]}
scorer = make_scorer(score_func)
rf = RandomForestRegressor()
search = GridSearchCV(rf, parameters)#,scoring=scorer)
result = search.fit(X_train, y_train.ravel())
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)
sorted(clf.cv_results_.keys())
scorer = make_scorer(rmsle, greater_is_better=False, size=10)
grid = GridSearchCV(est, param_grid, scoring=scorer)

from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators':[i for i in range(100,1100,100)],'max_depth':[i for i in range(1,21,2)],
              'min_samples_split':[i for i in range(2, 12)],'random_state': [42]}

rf = RandomForestRegressor()

#search = GridSearchCV(rf, parameter, scoring='accuracy', n_jobs=-1, cv=cv)
clf = GridSearchCV(rf, parameters)
clf.fit(X_train, y_train.ravel())
sorted(clf.cv_results_.keys())